# Анализ NPS клиентов телекоммуникационной компании

## Описание:

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».
Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite. 
Чтобы оценить результаты опроса, оценки обычно делят на три группы:
9-10 баллов — «cторонники» (англ. promoters);
7-8 баллов — «нейтралы» (англ. passives);
0-6 баллов — «критики» (англ. detractors).

## **Презентация и дашборд с выводами:**

<div class="alert alert-info">

[Презентация](https://disk.yandex.kz/d/4eLzDs7m4r2IJw)
    
[Дашборд](https://public.tableau.com/views/NPSTelecom/Dashboard1?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link)

In [1]:
#импорт библиотек
import os
import pandas as pd
from sqlalchemy import create_engine

**Подключимся к базе и  выгрузим данные**

In [2]:
# путь к БД на вашем компьютере
path_to_db_local = 'telecomm_csi.db'
# путь к БД на платформе
path_to_db_platform = '/datasets/telecomm_csi.db'
# итоговый путь к БД
path_to_db = None

# если путь на вашем компьютере ведёт к БД, то он становится итоговым
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
# иначе: если путь на платформе ведёт к БД, то он становится итоговым
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
# иначе выводится сообщение о том, что файл не найден
else:
    raise Exception('Файл с базой данных SQLite не найден!')
    
# если итоговый путь не пустой
if path_to_db:
    # то создаём подключение к базе
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)
    
    #запрос
    query = """
    SELECT user_id, /* */ 
           lt_day,
           CASE 
               WHEN lifetime.bucket_min <= 12 THEN 'new'
               ELSE 'old' 
           END AS is_new,
           SUBSTRING(age.title, 3, 10) AS age,
           CASE
               WHEN gender_segment = 1 THEN 'female'
               ELSE 'male'
           END gender_segment,
           os_name,
           cpe_type_name,
           country,
           city,
           SUBSTRING(age.title, 3, 10) AS age_segment,
           SUBSTRING(traffic.title, 3, 10) AS traffic_segment,
           SUBSTRING(lifetime.title, 3, 10) AS lifetime_segment,
           nps_score,
           CASE 
               WHEN nps_score <= 6 THEN 'критики'
               WHEN nps_score <= 8 THEN 'нейтралы'
               WHEN nps_score <= 10 THEN 'cторонники'
           END AS nps_group 
    FROM user
    LEFT OUTER JOIN age_segment AS age ON user.age_gr_id = age.age_gr_id
    LEFT OUTER JOIN traffic_segment AS traffic ON user.tr_gr_id = traffic.tr_gr_id
    LEFT OUTER JOIN lifetime_segment AS lifetime ON user.lt_gr_id = lifetime.lt_gr_id
    LEFT OUTER JOIN location ON user.location_id = location.location_id
    WHERE gender_segment IS NOT NULL;
    """
    # создаём датафрейм по данным запроса
    data = pd.read_sql(query, engine) 
    data.to_csv('project_2.csv', index = False)

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
data.info() #check

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501192 entries, 0 to 501191
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           501192 non-null  object
 1   lt_day            501192 non-null  int64 
 2   is_new            501192 non-null  object
 3   age               501192 non-null  object
 4   gender_segment    501192 non-null  object
 5   os_name           501192 non-null  object
 6   cpe_type_name     501192 non-null  object
 7   country           501192 non-null  object
 8   city              501192 non-null  object
 9   age_segment       501192 non-null  object
 10  traffic_segment   501192 non-null  object
 11  lifetime_segment  501192 non-null  object
 12  nps_score         501192 non-null  int64 
 13  nps_group         501192 non-null  object
dtypes: int64(2), object(12)
memory usage: 53.5+ MB


In [5]:
data

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old,45-54,female,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,cторонники
1,A001WF,2344,old,45-54,male,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,cторонники
2,A003Q7,467,old,55-64,male,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,cторонники
3,A004TB,4190,old,35-44,female,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,cторонники
4,A004XT,1163,old,16-24,male,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,cторонники
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501187,ZZZKLD,1249,old,45-54,female,ANDROID,SMARTPHONE,Россия,Москва,45-54,1-5,36+,5,критики
501188,ZZZLWY,129,new,25-34,male,ANDROID,SMARTPHONE,Россия,Москва,25-34,1-5,4-6,8,нейтралы
501189,ZZZQ5F,522,old,35-44,male,ANDROID,SMARTPHONE,Россия,Сургут,35-44,25-30,13-24,10,cторонники
501190,ZZZQ8E,2936,old,35-44,female,ANDROID,SMARTPHONE,Россия,УланУдэ,35-44,65-70,36+,9,cторонники


In [6]:
data['gender_segment'].value_counts()

female    272442
male      228750
Name: gender_segment, dtype: int64